@Abdulrahman Alabrash 

https://github.com/alabrashJr/DCNN-Julia


Parse each line trec_all_parsed.txt file to node struct

In [3]:
;head Data/TREC_all_parsed.txt

[advmod(develop-4, How-1), aux(develop-4, did-2), nsubj(develop-4, serfdom-3), root(ROOT-0, develop-4), prt(develop-4, in-5), cc(develop-4, and-6), advmod(leave-8, then-7), conj(develop-4, leave-8), dobj(leave-8, Russia-9), punct(develop-4, ?-10)]
[det(films-2, What-1), nsubj(featured-3, films-2), root(ROOT-0, featured-3), det(character-5, the-4), nsubj(Doyle-7, character-5), nn(Doyle-7, Popeye-6), xcomp(featured-3, Doyle-7), punct(featured-3, ?-8)]
[advmod(find-4, How-1), aux(find-4, can-2), nsubj(find-4, I-3), root(ROOT-0, find-4), det(list-6, a-5), dobj(find-4, list-6), prep(list-6, of-7), poss(names-11, celebrities-8), possessive(celebrities-8, '-9), amod(names-11, real-10), pobj(of-7, names-11), punct(find-4, ?-12)]
[det(fowl-2, What-1), nsubj(grabs-3, fowl-2), root(ROOT-0, grabs-3), det(spotlight-5, the-4), dobj(grabs-3, spotlight-5), prep(grabs-3, after-6), det(Year-9, the-7), amod(Year-9, Chinese-8), pobj(after-6, Year-9), prep(Year-9, of-10), det(Monkey-12, the-11), pobj(of-10

In [4]:
using Pkg; for p in ("JLD2","FileIO"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using HDF5,JLD2,FileIO

┌ Info: Recompiling stale cache file /home/ec2-user/.julia/compiled/v1.1/HDF5/L7Dga.ji for HDF5 [f67ccb44-e63f-5c2f-98bd-6dc0ccc4ba2f]
└ @ Base loading.jl:1184


In [5]:
mutable struct node 
    word
    kidsword
    kidsindex
    parent
    finished
    is_word
    selfindex
    parentindex
    label
    ind
node(word) = word == nothing ? new(nothing,nothing,nothing,nothing,nothing,0,nothing,nothing,nothing,nothing) : new(word,[],[],[],0,1,0,0,"",-1)
    
end

In [6]:
input_file  = "Data/TREC_all_parsed.txt";

In [7]:
input = open(input_file,"r")
counter =0
doc = []
fileReaded=readlines(input)
for i in fileReaded
           counter += 1
            #counter
           #i = strip(i)
           i = strip(i,['[',']',' '])
           #print i
           current = split(i,"), ")
           node_container = Dict()
           ROOT = node("ROOT")
           node_container[0] = ROOT    
           for (index,j) in enumerate(current)
               
               label = split(j,'(')[1]
               j = strip((split(j,'(')[2]),')')
               current_list = [strip(i) for i in split(j,", ")]
               current_node=node(join(split(current_list[2],'-')[1:end-1],'_'))
               current_node.selfindex=parse(Int,split(current_list[2],'-')[end])
               current_node.label = label
               if join(split(current_list[1],'-')[1:end-1],'_') == "ROOT"
                       current_node.parent = ROOT
               else               
                    current_node.parent=join(split(current_list[1],'-')[1:end-1],'_')
                    current_node.parentindex=parse(Int,split(current_list[1],'-')[end])
               end    
               node_container[current_node.selfindex] = current_node
           
           end
           node_container1 = deepcopy(node_container);
            for (index,current) in node_container1
               p_index = current.parentindex
               if p_index ∉ keys(node_container)
                   new_node = node(current.parent)
                   new_node.parent = ROOT
                   new_node.selfindex = p_index
                   node_container[p_index] = new_node    
               end
            end
           
            for (index,current) in node_container1
               if current.word == "ROOT"
                   continue
               end
               p_index = current.parentindex
               #print p_index
               if p_index ∉ keys(node_container)
                   new_node = node(current.parent)
                   new_node.parent = ROOT
                   new_node.selfindex = p_index
                   node_container[p_index] = new_node
               end
               p = node_container[p_index]
               
               push!(p.kidsword,current.word)
               push!(p.kidsindex,current.selfindex)
           end
   #         for (j,i) in node_container
   #             println( "self\t", i.selfindex, "\tparent\t", i.parentindex,"\tchild\t", i.kidsindex)
          # end
           push!(doc,node_container)
           #break 
end

In [8]:
save("Data/data.jld2","data",doc) # using FileIO pacakge 

In [ ]:
#to do change JLD to JLD2 https://github.com/JuliaIO/JLD2.jl